The main goal of this document will be to explore polynomial dynamical systems on the integers modulo $N$. That is, consider a polynomial $f(x)$, and view it as a dynamical system on $\mathbb{Z}/N\mathbb{Z}$ by iteratively applying it. What can we say, for instance, about the resulting orbits?

In [ ]:
import matplotlib.pyplot as plt
import math

The simplest polynomials that we can analyse are $f(x) = x + n$ for some integer constant $n$. It doesn't take much thought to understand its orbits modulo $N$. But now let's consider polynomials such as $f(x) = 2x$. What happens in that case?

In [ ]:
def stable_orbit(f, N, x):
    """Given a (say polynomial) function f, a number N, and a starting value x, we iteratively apply x = f(x) mod N until we end up in a loop. We return that loop."""
    x = x % N
    values = [x]
    for _ in range(N):
        x = f(x) % N
        try:
            #If we find an x that has already been obtained previously, then we have obviously found the stable orbit.
            find = values.index(x)
            return values[find :: ]
        except ValueError:
            values.append(x)
    return []

In [ ]:
def f(x):
    return 2*x

max_value = 1000
data = [len(stable_orbit(f, N, 1)) for N in range(1, max_value + 1)]

In [ ]:
plt.scatter(range(1, max_value + 1), data, s = 2)
plt.show()

That's a pretty cool plot. One thing we see is that there are some obvious lines appearing in this graph. Indeed there are numerous $N$ for which the length of the stable orbit is equal to $N - 1$:

In [ ]:
max_orbits = [i + 1 for i in range(max_value) if data[i] == i]
print(max_orbits)

With the exception of $N = 2$, a number is in this list if and only if $N$ is prime and $2$ is a primitive root mod $N$. It is a famous conjecture of Artin that there are infinitely many such prime numbers. This conjecture is known to follow from the generalised Riemann hypothesis. With that in mind, it's safe to say that we know just as little about the lines with other slopes in the plot.

One additional thing that stands out is that there are some random values lying in between the line of slope $1$ and the line of slope $1/2$. These turn out to be prime powers. For instance, if $N = 625 = 5^4$, then we find the length of the stable period to be $500$, while if $N = 729 = 3^6$, then the length is $486$.

Now, what we can say about the number of stable orbits? That is, instead of starting at $0$, consider all starting values and look at their stable orbits.

In [ ]:
def count_stable_orbits(f, N):
    """Count the number of stable orbits of the function x -> f(x) % N."""
    found_in_orbit = {x : False for x in range(N)} #We keep track of those numbers for which we already know what stable orbit it lands into. This avoids quite a bit of redundancy.
    count = 0
    for x in range(N):
        if not found_in_orbit[x]:
            values = [x]
            while True: 
                x = f(x) % N
                if found_in_orbit[x]:
                    break
                elif x in values:
                    for value in values:
                        found_in_orbit[value] = True
                    count += 1
                    break
                else:
                    values.append(x)
    return count

This is a much more data-intensive operation, since, at least in principle, we can have to consider every starting value. The code has been optimised to avoid redundancy as much as I can, but it remains painful.

In [ ]:
max_value = 1000
data = [count_stable_orbits(f, N) for N in range(1, max_value + 1)]

In [ ]:
plt.scatter(range(1, max_value + 1), data, s = 2)
print("First couple of values:", data[:21])
plt.show()

The sequence can be found in OEIS as A000374. It's mentioned that it is equal to the number of irreducible factors in the factorization of the polynomial $x^N - 1$ over $\mathbb{F}_2$, which is pretty cool.

One thing worth noting is that we don't really need to make use of the mulitplicative structure of the integers modulo $N$ when defining this dynamical system. That is, instead of writing $x \mapsto 2x$, we can write $x \mapsto x + x$. It's tempting to ask how this system behaves over general monoids. Notably, when applied to the multiplicative units of $\mathbb{Z} / p\mathbb{Z}$, we would get the behaviour of the polynomial $f(x) = x^2$. In a separate file I will use GAP to investigate the behaviour of this system.

The observation above completely break down if instead we consider a system that somehow mixes up addition and multiplication. So let's consider a polynomial like $f(x) = x^2 + 1$, with starting value $0$, and let's just see where that gets us.

In [ ]:
def f(x):
    return x**2 + 1

for N in range(1, 41):
    print(stable_orbit(f, N, 0))

That seems elegantly chaotic! What happens when we just look at the lengths of the stable orbits?

In [ ]:
max_value = 100000
data = [len(stable_orbit(f, N, 0)) for N in range(1, max_value + 1)]

In [ ]:
plt.scatter(range(1, max_value + 1), data, s = 2)
plt.show()

It may look a little better on a log-plot.

In [ ]:
log_data = [math.log(n) for n in data]
plt.scatter(range(1, max_value + 1), log_data, s = 2)
plt.show()

I don't know what one can say about the growth of this function in general. Let's denote the function by $L(N)$. It's not hard to show that $L$ commutes with least common multiples, in that

$$ L(\operatorname{lcm}(M,N)) = \operatorname{lcm}(L(M), L(N)). $$

There doesn't seem to be any nontrivial lower bound, because even for large $N$, there seem to be many stable periods of length $1$. That this can happen is not too surprising; indeed, there are numerous solutions to the equation $x = x^2 + 1$ mod $N$ in general. Still, the infinitude of $N$ such that $L(N) = 1$ is not obvious to me, because it's not clear when exactly the orbit of $0$ will reach such a value.

What about an upper bound? Surely there should be something that we can say, right? For trivial reasons, we know that $L(N) \leq N$. How strong is this bound?

In [ ]:
quotient_data = [data[n]/(n + 1) for n in range(max_value)]
plt.scatter(range(1, max_value + 1), quotient_data, s = 2)
plt.show()

I'm quite surprised at how 'close to sharp' this upper bound already is. Yes, most values of $L(N)$ are much less than $N$, but there are some obvious occasional outliers. Even at $N = 75\,000$ one finds values for which $L(N) / N$ is above $0.1$. 

What if we count the number of stable orbits in general? That is, instead of starting at $0$, consider all starting values and look at their stable orbits.

In [ ]:
def count_stable_orbits(f, N):
    """Count the number of stable orbits of the function x -> f(x) % N."""
    found_in_orbit = {x : False for x in range(N)} #We keep track of those numbers for which we already know what stable orbit it lands into. This avoids quite a bit of redundancy.
    count = 0
    for x in range(N):
        if not found_in_orbit[x]:
            values = [x]
            while True: 
                x = f(x) % N
                if found_in_orbit[x]:
                    break
                elif x in values:
                    for value in values:
                        found_in_orbit[value] = True
                    count += 1
                    break
                else:
                    values.append(x)
    return count

This is a much more data-intensive operation, since, at least in principle, we can have to consider every starting value. The code has been optimised to avoid redundancy, but it is still rather slow. Since I plan to submit this sequence to the OEIS, I will upload a faster script written in C in the near future.

In [ ]:
max_value = 1000
data = [count_stable_orbits(f, N) for N in range(1, max_value + 1)]

In [ ]:
plt.scatter(range(1, max_value + 1), data, s = 2)
plt.show()

Can we say anything about the *sizes* of the various stable orbits?

In [ ]:
def stable_orbit_sizes(f, N):
    """Count the siezs of the stable orbits of the function x -> f(x) % N."""
    found_in_orbit = {x : False for x in range(N)} #We keep track of those numbers for which we already know what stable orbit it lands into. This avoids quite a bit of redundancy.
    sizes = []
    for x in range(N):
        if not found_in_orbit[x]:
            values = [x]
            while True: 
                x = f(x) % N
                if found_in_orbit[x]:
                    break
                else:
                    try:
                        find = values.index(x)
                        sizes.append(len(values) - find)
                        for value in values:
                            found_in_orbit[value] = True
                        break
                    except ValueError:
                        values.append(x)
    return sizes

Let's take some random values in some arbitrary interval:

In [ ]:
def f(x):
    return x**2 + 1

for N in range(20240, 20280):
    print("N =", N, ":", stable_orbit_sizes(f, N))

I find it incredible how much repetition occurs in the periods of the orbits, not to mention how many orbit lengths are scalar multiples of each other. It strongly suggests the presence of a certain symmetry governing the dynamical system. What might that symmetry be?